# Plotting dark matter box coordinates in 3d, to get an idea of the  selected region

Running this requires the `read_eagle_files` routine that can be found here: https://gitlab.cosma.dur.ac.uk/jch/Read_Eagle

In [1]:
%matplotlib nbagg
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import read_eagle_files 
import pandas as pd
import gc


h = 0.6777 #Hubble parameter being used in the simulations
boxsize = 2168.64 #Boxsize in h^-1 units

/cosma/local/Python/2.7.3/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def create_3d(coords):

    # Do the plotting in a single call.
    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d')
    ax.scatter(coords[:,0], coords[:,1], coords[:,2], s = 0.1)
    ax.view_init(elev=8, azim=17)
    return fig, ax
    

In [3]:
# Returns the position of particles within a box of side length 2 x dist, given the 
# center. Option turn on periodic box corrections.

def get_coords(coords, dist, per_corr = 'off'):
    
    grp_centre = coords
    
    #Should be in h^-1 cMpc
    xpos = grp_centre[0]
    ypos = grp_centre[1]
    zpos = grp_centre[2]

    xmin = xpos - (dist)
    xmax = xpos + (dist)
    ymin = ypos - (dist)
    ymax = ypos + (dist)
    zmin = zpos - (dist)
    zmax = zpos + (dist)

    box = [xmin, xmax, ymin, ymax, zmin, zmax]

    
    snap_dir = '/cosma5/data/dp004/PROJECTS/Eagle/dm_eagle_volume_L3200/snapdir_002/snapshot_L3200_002.0.hdf5'
    snap = read_eagle_files.read_eagle.EagleSnapshot(snap_dir)
    
    #Give values in terms of Mpc
    snap.select_region(box[0], box[1], box[2], box[3], box[4], box[5])
    pos = snap.read_dataset(1, "Coordinates")
    
    coords_temp = pos - grp_centre
    
    #Correcting for the periodic box if the points are close to the boundary
    
    if per_corr == 'on':
    
        ok = np.where(coords_temp[:,0]>=boxsize/2.)
        coords_temp[:,0][ok] -= boxsize

        ok = np.where(coords_temp[:,1]>=boxsize/2.)
        coords_temp[:,1][ok] -= boxsize

        ok = np.where(coords_temp[:,2]>=boxsize/2.)
        coords_temp[:,2][ok] -= boxsize

        ok = np.where(coords_temp[:,0]<=-boxsize/2.)
        coords_temp[:,0][ok] += boxsize

        ok = np.where(coords_temp[:,1]<=-boxsize/2.)
        coords_temp[:,1][ok] += boxsize

        ok = np.where(coords_temp[:,2]<=-boxsize/2.)
        coords_temp[:,2][ok] += boxsize

        tmp = np.sum((coords_temp)**2, axis=1)**0.5
        ok = np.where(tmp <= 10.)
    
    
    gc.collect()
    
    return coords_temp[ok]

In [4]:
def plot_region(center, delta, name):
    
    pos = get_coords(center, dist = 11)
    print 'Number of DM particles: ', (len(pos))
    fig, axs = create_3d(pos)
    axs.set_title(r'$\delta$ = {}, center = ({},{},{})'.format(delta, center[0], center[1], center[2]))
    plt.savefig('{}.png'.format(name))
    plt.show()
